<h1 align="center">Bases de Datos con Python y PostgreSQL</h1> 

PostgreSQL es un sistema de gestión de bases de datos relacionales (RDBMS) de código abierto, que ofrece una gran cantidad de características y opciones para el almacenamiento y procesamiento de datos. Es muy popular entre los desarrolladores y se utiliza en una amplia variedad de aplicaciones, desde pequeñas aplicaciones de escritorio hasta grandes sistemas de gestión de datos empresariales.

### Conexión a Base de Datos

Para conectarte a una base de datos PostgreSQL desde Python, puedes utilizar la biblioteca psycopg2. Para instalarla, puedes utilizar el siguiente comando en una terminal:

In [ ]:
pip install psycopg2-binary

A continuación, importa el paquete psycopg2 y se establece una conexión a la base de datos de PostgreSQL usando los detalles de conexión, como el host, la base de datos, el usuario y la contraseña.

In [ ]:
import psycopg2

conexion = psycopg2.connect(
    user="mi_usuario",
    password="mi_contraseña",
    host='127.0.0.1',
    port='5432',
    database="mi_base_de_datos"
)

print(conexion) #<connection object at 0x0000020747C67890; dsn: 'user=postgres password=xxx dbname=Test_db host=127.0.0.1 port=5432', closed: 0>

Una vez concetado, es importante tener presente ciertas consultas básicas usadas. Es recomendable usar palabras reservadas en mayúscula. 
En este caso, "persona" es la base de datos creada.

-Para recuperar información de la base de datos se usa:

SELECT * FROM persona

-Para limitar la cantidad de registros a ser mostrados:

SELECT * FROM persona WHERE id_persona IN (rango)

-Para insertar información:

INSERT INTO persona(columnas a afectar separadas por ,) VALUES (valores que entrarán en las columnas entre '' y separadas por ,)
Esto último, sigue un órden, por ende el primer valor de VALUES se asociará al primer valor de persona().

-Para actualizar alguno de los registros:

UPDATE persona SET 'columna a ser cambiada' = 'Nuevo nombre', 'otra columna' = 'xx' WHERE  id_persona=(rango)
En este caso, la columna a ser cambiada no lleva comillas, pero lo que va a modificar sí, como también es importante indicar a qué registros se les va a hacer la actualización o se actualizarán todos.

-Para eliminar un registro:

DELETE FROM persona WHERE id_persona=(rango)
En este caso también es importante señalar el "where" de caso contrario se eliminará todo.

Como siguiente paso, creamos un cursor, a través del cual podemos ejecutar consultas SQL:

In [ ]:
cursor = conexion.cursor()
sentencia = 'SELECT * FROM persona'
cursor.execute(sentencia) #Ejecuta la sentencia
registros = cursor.fetchall() #Obtiene todos los registros
print(registros) #Ej: [('Juan', 35), ('Karla', 37), ('Ricardo', 20)]

cursor.close()
conexion.close() #Cierra la conexión

### Uso de Python y PostgreSQL

El uso de with es una buena práctica ya que asegura que la conexión y el cursor se cierren correctamente al final de su uso, evitando así posibles errores o fugas de memoria.

In [ ]:
import psycopg2

conexion = psycopg2.connect(
    user="mi_usuario",
    password="mi_contraseña",
    host='127.0.0.1',
    port='5432',
    database="mi_base_de_datos"
)

with conexion:
    with conexion.cursor() as cursor:
        sentencia = 'SELECT * FROM persona'
        cursor.execute(sentencia)
        registros = cursor.fetchall()
        print(registros)

De esta manera, no hace falta usar el bloque de la conexión, ya que de eso se encarga el bloque with, sin embargo, es recomedable encasillar todo en un bloque try con finally para cerrar el objeto conexión.

In [ ]:
import psycopg2

conexion = psycopg2.connect(
    user="mi_usuario",
    password="mi_contraseña",
    host='127.0.0.1',
    port='5432',
    database="mi_base_de_datos"
)
try:
    with conexion:
        with conexion.cursor() as cursor:
            sentencia = 'SELECT * FROM persona'
            cursor.execute(sentencia)
            registros = cursor.fetchall()
            print(registros)
except Exception as e:
    print(f'Ocurrió un error: {e}')
finally:
    conexion.close()

Por otra parte, para regresar un solo registro, teniendo en cuenta que fetchall recupera todo, lo que hacemos es cambiar la sentencia con las consultas básicas ya vistas

sentencia = 'SELECT * FROM persona WHERE id_persona = 2'

Esto puede ser más dinámico al usar parámetros denominados placeholder o parámetro posicional y se usa llamando '%s', para así sustituir el valor a ser llamado por una variable. En el ejemplo siguiente, se le pide al usuario que proporcione lo que quiera recuperar.

In [ ]:

try:
    with conexion:
        with conexion.cursor() as cursor:
            sentencia = 'SELECT * FROM persona WHERE id_persona = %s'
            entrada = input('Proporciona el id_persona: ') #Variable a ser llamada
            cursor.execute(sentencia, (entrada,)) #Se debe enviar una tupla
            registros = cursor.fetchall()
            print(registros)
except Exception as e:
    print(f'Ocurrió un error: {e}')
finally:
    conexion.close()

Otro valor a tener en cuenta, es 'fetchone' que es un método que se utiliza para obtener una única fila de resultados de la consulta. Cada vez que se llama al método fetchone, se obtiene una nueva fila hasta que no haya más filas que recuperar. Esto es especialmente útil cuando se espera un conjunto grande de resultados y se quiere procesarlos uno a uno para ahorrar memoria.

A diferencia con fetchall es que este es un método que se utiliza para obtener todas las filas que devuelve una consulta. Es decir, devuelve una lista con todas las filas obtenidas como resultado de la consulta. Este método es útil cuando se espera un conjunto relativamente pequeño de resultados.

Por lo que  la principal diferencia entre fetchall y fetchone es que fetchall devuelve todas las filas obtenidas por la consulta como una lista, mientras que fetchone devuelve una sola fila cada vez que se llama al método.

Ahora para insertar un registro, usamos la sintaxis ya mencionada pero cambiando la sentencia con las mencionadas anteriormente.

In [ ]:
import psycopg2

conexion = psycopg2.connect(
    user="mi_usuario",
    password="mi_contraseña",
    host='127.0.0.1',
    port='5432',
    database="mi_base_de_datos"
)
try:
    with conexion:
        with conexion.cursor() as cursor:
            sentencia = 'INSERT INTO persona(nombre, apellido, mail) VALUES(%s, %s, %s)'
            valores = (
                ('Jorge', 'Buitrago', 'jbui@mail.com'),
                ('Angie', 'Alonso', 'alonso@mail.com'),
                ('Roberto', 'Carlos', 'rcarlos@mail.com')
            )
            cursor.executemany(sentencia, valores)
            registros_insertados = cursor.rowcount
            print(f'Registros insertados: {registros_insertados}')  #Registros insertados: 3
except Exception as e:
    print(f'Ocurrió un error: {e}')
finally:
    conexion.close()

En el código anterior, como se está insertando información, a diferencia de las consultas básicas, se debe usar el método .commit() para guardar los cambios en la base de datos, en este caso no ya que el método 'with' lo hace. También usamos .rowcount para saber cuántas filas se han insertado y se cambia excute por executemany para insertar varios registros a la vez.

Para actualizar registros, se usa la misma sintaxis que para insertar, pero se cambia la sentencia por UPDATE y se agrega el WHERE para indicar qué registros se van a actualizar.

In [ ]:
import psycopg2

conexion = psycopg2.connect(
    user="mi_usuario",
    password="mi_contraseña",
    host='127.0.0.1',
    port='5432',
    database="mi_base_de_datos"
try:
    with conexion:
        with conexion.cursor() as cursor:
            sentencia = 'UPDATE persona SET nombre = %s, apellido = %s, mail = %s WHERE id_persona = %s'
            valores = ('Andrés', 'Gómez', 'agomez@mail.com', 4)
            cursor.execute(sentencia, valores)
            registros_actualizados = cursor.rowcount
            print(f'Registros actualizados: {registros_actualizados}')  #Registros insertados: 1
except Exception as e:
    print(f'Ocurrió un error: {e}')
finally:
    conexion.close()

No es necesario modificar todas las columnas, se puede modificar una sola columna o varias, pero siempre se debe indicar el WHERE para indicar qué registros se van a actualizar. Para actualizar varios registros a la vez, se usa el método executemany.

Para eliminar registros, se usa la misma sintaxis que para actualizar, pero se cambia la sentencia por DELETE y se agrega el WHERE para indicar qué registros se van a eliminar.

In [ ]:
import psycopg2

conexion = psycopg2.connect(
    user="mi_usuario",
    password="mi_contraseña",
    host='127.0.0.1',
    port='5432',
    database="mi_base_de_datos"
)
try:
    with conexion:
        with conexion.cursor() as cursor:
            sentencia = 'DELETE FROM persona WHERE id_persona = %s'
            valores = (1,)
            cursor.execute(sentencia, valores)
            registros_eliminados = cursor.rowcount
            print(f'Registros eliminados: {registros_eliminados}')  #Registros insertados: 1
except Exception as e:
    print(f'Ocurrió un error: {e}')
finally:
    conexion.close()

Para eliminar varios registros a la vez, se usa el método executemany y recordar que es importante la tupla de tuplas. Ej: ((1,), (2,))

### Manejo de Transacciones

 Las transacciones son un mecanismo para agrupar varias operaciones de bases de datos en una sola unidad lógica. Si una transacción completa con éxito todas sus operaciones, los cambios se aplican a la base de datos de manera permanente. Si una transacción falla, los cambios se revierten y la base de datos vuelve al estado anterior a la transacción.

In [ ]:
import psycopg2

conexion = psycopg2.connect(
    user="mi_usuario",
    password="mi_contraseña",
    host='127.0.0.1',
    port='5432',
    database="mi_base_de_datos"
)
try:
    conexion.autocommit = False

    cursor = conexion.cursor()
    sentencia = 'INSERT INTO persona(nombre, apellido, mail) VALUES(%s, %s, %s)'
    valores = ('Carlos', 'Lara', 'clara@mail.com')
    cursor.execute(sentencia, valores)

    sentencia = 'UPDATE persona SET nombre = %s, apellido = %s, mail = %s WHERE id_persona = %s'
    valores = ('Juan', 'Perez', 'jp@mail.com', 1)
    cursor.execute(sentencia, valores)

    conexion.commit()
    print('Termina la transacción')
except Exception as e:
    conexion.rollback()
    print(f'Ocurrió un error: {e}')
finally:
    conexion.close()

En este caso tenemos dos conceptos a tener el cuenta 'commit' y 'rollback'. Donde el commit confirma la transacción y el rollback la cancela. También se indica que los cambios no se guarden automáticamente, sino que se debe usar el método commit() para guardar los cambios en la base de datos, esto para tener un mayor control sobre las transacciones. Como anotación, si no se especifica el autocommit, por defecto se encuentra desactivado. También se puede ahorrar la configuración manual de commit, delegandolo al manejo de recursos 'with' y así se ejecuta el commit automáticamente al finalizar el bloque with. 

In [ ]:
import psycopg2

conexion = psycopg2.connect(
    user="mi_usuario",
    password="mi_contraseña",
    host='127.0.0.1',
    port='5432',
    database="mi_base_de_datos"
)
try:
    with conexion:
        with conexion.cursor() as cursor:
            sentencia = 'INSERT INTO persona(nombre, apellido, mail) VALUES(%s, %s, %s)'
            valores = ('Carlos', 'Lara', 'clara@mail.com')
            cursor.execute(sentencia, valores)

            sentencia = 'UPDATE persona SET nombre = %s, apellido = %s, mail = %s WHERE id_persona = %s'
            valores = ('Juan', 'Perez', 'jp@mail.com', 1)
            cursor.execute(sentencia, valores)
except Exception as e:
    print(f'Ocurrió un error: {e}')
finally:
    conexion.close()

### Pool de Conexiones

Cuando trabajamos con bases de datos en Python, es común utilizar la biblioteca psycopg2 para conectarnos a una base de datos PostgreSQL. Sin embargo, cada vez que se realiza una conexión a la base de datos, se crea una nueva conexión y se cierra al finalizar la consulta. Si se realizan muchas consultas en un corto período de tiempo, esto puede llevar a una sobrecarga del servidor y ralentizar el rendimiento.

Para evitar esto, se puede utilizar una pool de conexiones, que permite reutilizar las conexiones ya establecidas en lugar de crear nuevas conexiones cada vez.

In [ ]:
from psycopg2 import pool
import sys

class Conexion:
    _DATABASE = 'Test_db'
    _USERNAME = 'postgres'
    _PASSWORD = 'Xevaxtiam1'
    _DB_PORT = '5432'
    _HOST = '127.0.0.1'
    _minconn = 1
    _maxconn = 5
    _pool = None

    @classmethod
    def obtenerPool(cls):
        if cls._pool is None:
            try:
                cls._pool = pool.SimpleConnectionPool(cls._minconn, cls._maxconn,
                                                      host=cls._HOST,
                                                      user=cls._USERNAME,
                                                      password=cls._PASSWORD,
                                                      port=cls._DB_PORT,
                                                      database=cls._DATABASE) 
                print(f'Pool exitoso: {cls._pool}')
                return cls._pool
            except Exception as e:
                print(f'Ocurrió un error al obtener el pool: {e}')
                sys.exit()
        else:
            return cls._pool

    @classmethod
    def obtenerConexion(cls):
        conexion = cls.obtenerPool().getconn()
        print(f'Conexión exitosa: {conexion}')
        return conexion
        
    @classmethod
    def liberarConexion(cls, conexion):
        cls.obtenerPool().putconn(conexion)
        print(f'Conexión liberada: {conexion}')

    @classmethod
    def cerrarConexiones(cls):
        cls.obtenerPool().closeall()
        print(f'Conexiones cerradas')

En el ejemplo anterior, creamos una pool de conexiones con un mínimo de 1 conexión y un máximo de 10 conexiones. También proporcionamos los detalles de conexión a la base de datos, como el nombre de usuario, la contraseña, la dirección del servidor, el puerto, el nombre de la base de datos y con SimpleConnectionPool() se crea la pool de conexiones. También las líneas necesarias para liberar la conexión al finalizar la consulta, cerrar la conexión. 